# Fetch guternberg books from a category

## Step 1, get book ids

- go to http://m.gutenberg.org/ebooks/search.mobile/?query=bsxHorror&sort_order=downloads

- scroll to the bottom and click "show more" a few times
- enter the javascript below in the browsers js console
- it should have copied the ids to your clipboard, you can paste it into "ids" below


```js
// to get all book ids shown on page, paste this javascript into js console in browser when on the page above
a_elems = document.getElementsByClassName("table link")
hrefs = Array.from(a_elems)
  .map(e=>e.href) // get link
  .filter(e=>e) // remove empty links
ids = hrefs.map(e=>/(\d+)\.mobile/.exec(e)) // regular expression match
  .filter(e=>e) // remove ones not found
  .map(e=>e[1]) // get just id
copy(ids) // copy to clipboard
```

In [1]:
import os
os.sys.path.append('..')

In [2]:
import requests
import os
import re
import bs4
import time
import json


dest_dir = '../data/input/erotica_gutenberg'
if not os.path.isdir(dest_dir):
    os.makedirs(dest_dir)
    
raw_dir = os.path.join(dest_dir, 'raw')
if not os.path.isdir(raw_dir):
    os.makedirs(raw_dir)

In [3]:
from tqdm import tqdm as tqdm

In [4]:
from dataset import parse_gutenberg

In [5]:
# urls to download text inputs
ids = [
    "30254",
    "30360",
    "28520",
    "25305",
    "14005",
    "28522",
    "31284",
    "28521",
    "29827",
    "52059",
    "14323",
    "13610",
    "57284",
    "13972",
    "52205",
    "54672",
    "13614",
    "28718",
    "44877",
    "26804",
    "45150",
    "37491",
    "43438",
    "48943",
    "53807",
    "26456",
    "26808",
    "13971",
    "42406",
    "43823",
    "39220",
    "56779",
    "26809",
    "18610",
    "44181",
    "42212",
    "26806",
    "42586",
    "47892",
    "43822",
    "49855",
    "26562",
    "26739",
    "26807",
    "20568",
    "40877",
    "54419",
    "53944",
    "40557",
    "29049",
    "25543",
    "40902",
    "41301",
    "56491",
    "28789",
    "40496"
]


In [7]:
# download/cache raw files
for bid in tqdm(ids, mininterval=60):
    
    # first download index
    index_url = "http://www.gutenberg.org/files/{bid:}".format(bid=bid)
    r = requests.get(index_url)
    r.raise_for_status()
    soup = bs4.BeautifulSoup(r.content, "html5lib")
    hrefs = [e.attrs['href'] for e in soup.findAll('a')]
    links = [h for h in hrefs if h.endswith('.txt')]
    time.sleep(0.1) # avoid ddos/ban
    
    # download text
    for link in links:
        txt_url = index_url + '/' + link
        outfile = os.path.join(dest_dir, 'raw', link)
        if not os.path.isfile(outfile):
            r = requests.get(txt_url)
            r.raise_for_status()
            open(outfile, 'w').write(r.text)
            
            time.sleep(0.1) # avoid ddos/ban

100%|██████████| 56/56 [00:32<00:00,  1.72it/s]


# 2. turn into cleaned(ish) txt

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
# convert raw text into one long csv

max_len = 400
num_sent = 6
data=[]
for infile in os.listdir(raw_dir):
    path = os.path.join(raw_dir, infile)
    info = parse_gutenberg(open(path).read())
    if info['language']=='English':
        print(info['title'])
        data.append(info['content'])

The Ladies Delight
The Life and Amours of the Beautiful, Gay and Dashing Kate Percival, The Belle of the Delaware
The Erotic Motive in Literature
The Fifteen Comforts of Matrimony: Responses from Men
    Problem: No title found

    Problem: No '*** START' seen

    Problem: No '*** END' seen

The Power of Mesmerism, A Highly Erotic Narrative of Voluptuous Facts and Fancies
Memoirs Of Fanny Hill, A New and Genuine Edition from the Original Text (London, 1749)
Studies in the Psychology of Sex, Volume 5 (of 6)
Forbidden Fruit, Luscious and exciting story and More forbidden fruit or, Master Percy's progress in and beyond the domestic circle
La-bas
Studies in the Psychology of Sex, Volume 5 (of 6)
Là-bas
    Problem: No title found

    Problem: No '*** START' seen

    Problem: No '*** END' seen

The Power of Mesmerism, A Highly Erotic Narrative of Voluptuous Facts and Fancies
Les Bijoux Indiscrets, or, The Indiscreet Toys
Anthologica Rarissima: The Way of a Virgin
The Life and Amours of 

In [10]:
x_train, x_test = train_test_split(data)
x_val, x_test = train_test_split(x_test)

In [11]:
open(os.path.join(dest_dir, "train.txt"), "w").write("\n\n".join(x_train))
open(os.path.join(dest_dir, "val.txt"), "w").write("\n\n".join(x_val))
open(os.path.join(dest_dir, "test.txt"), "w").write("\n\n".join(x_test))

798250